#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/competitions/desafio-de-prediccion-de-precios-de-portatiles

## Librerias

In [2697]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ToolBox as TB
import re
from scipy import stats
import regex as re
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


## Data

In [2698]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
df_train = pd.read_csv ('Data/train.csv')
df_test = pd.read_csv ('Data/test.csv')

In [2699]:
print (df_train.shape)
print (df_test.shape)

(912, 14)
(391, 13)


## Data exploration

In [2700]:
df_train.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                912 non-null    int64  
 1   laptop_ID         912 non-null    int64  
 2   Company           912 non-null    object 
 3   Product           912 non-null    object 
 4   TypeName          912 non-null    object 
 5   Inches            912 non-null    float64
 6   ScreenResolution  912 non-null    object 
 7   Cpu               912 non-null    object 
 8   Ram               912 non-null    object 
 9   Memory            912 non-null    object 
 10  Gpu               912 non-null    object 
 11  OpSys             912 non-null    object 
 12  Weight            912 non-null    object 
 13  Price_euros       912 non-null    float64
dtypes: float64(2), int64(2), object(10)
memory usage: 99.9+ KB


In [2701]:
df_test.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                391 non-null    int64  
 1   laptop_ID         391 non-null    int64  
 2   Company           391 non-null    object 
 3   Product           391 non-null    object 
 4   TypeName          391 non-null    object 
 5   Inches            391 non-null    float64
 6   ScreenResolution  391 non-null    object 
 7   Cpu               391 non-null    object 
 8   Ram               391 non-null    object 
 9   Memory            391 non-null    object 
 10  Gpu               391 non-null    object 
 11  OpSys             391 non-null    object 
 12  Weight            391 non-null    object 
dtypes: float64(1), int64(2), object(10)
memory usage: 39.8+ KB


In [2702]:
df_train.describe ()

,id,laptop_ID,Inches,Price_euros
count,912.000000,912.000000,912.000000,912.000000
mean,645.091009,666.192982,15.011404,1108.122873
std,380.313813,384.873846,1.411744,714.597741
min,0.000000,1.000000,10.100000,174.000000
25%,319.750000,330.500000,14.000000,589.000000
50%,636.500000,673.500000,15.600000,949.000000
75%,980.250000,998.500000,15.600000,1458.500000
max,1302.000000,1320.000000,18.400000,6099.000000


In [2703]:
df_train.head ()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37


In [2704]:
TB.describe_df (df_train)

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
DATA_TYPE,int64,int64,object,object,object,float64,object,object,object,object,object,object,object,float64
MISSINGS (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UNIQUE_VALUES,912,912,19,475,6,15,35,104,8,36,91,9,158,614
CARDIN (%),100.0,100.0,2.08,52.08,0.66,1.64,3.84,11.4,0.88,3.95,9.98,0.99,17.32,67.32


In [2705]:
TB.tipifica_variables (df_train, 10, 30)

,nombre_variable,tipo_sugerido
0,id,Numérica Continua
1,laptop_ID,Numérica Continua
2,Company,Numérica Discreta
3,Product,Numérica Continua
4,TypeName,Categórica
5,Inches,Numérica Discreta
6,ScreenResolution,Numérica Discreta
7,Cpu,Numérica Discreta
8,Ram,Categórica
9,Memory,Numérica Discreta


In [2706]:
df_train.head ()

,id,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,1366x768,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R7 M445,Windows 10,2.36kg,749.00
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,1366x768,Intel Pentium Quad Core N4200 1.1GHz,4GB,1TB HDD,Intel HD Graphics 505,Windows 10,2kg,449.00
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1460.00
3,578,767,Dell,Alienware 17,Gaming,15.6,IPS Panel 4K Ultra HD 3840x2160,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1070,Windows 10,4.42kg,2868.99
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 7,1.26kg,1713.37


## Data processing

In [2707]:
print(df_test.columns)


Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight'],
      dtype='object')


In [2708]:
# Columna Cpu

def extract_from_Cpu(train):
    train["Vel"] = train["Cpu"].str.extract(r'(\d+(?:\.\d+)?GHz)')
    train["Vel"] = train["Vel"].str.replace('GHz','')
    train["Vel"] = train["Vel"].astype(float)
    train["Cpu"] = train["Cpu"].str.replace(r'(\d+(?:\.\d+)?GHz)','',regex =True)
    

extract_from_Cpu (df_test)
extract_from_Cpu (df_train)


In [2709]:
# Columna ScreenResolution

def extract_screen_features(train):
    #train = train.rename(columns = str.lower)
    train["resolucion"] = train['ScreenResolution'].str.extract(r'(\d+x\d+)')
    train["pantalla"]= train['ScreenResolution'].replace(r'(\d+x\d+)','', regex = True)
    train['pantalla'] = train['pantalla'].replace(r'(Full HD|Quad HD|Quad HD|\+|/|4K Ultra HD)','', regex = True)
    train['tactil'] = train['pantalla'].str.extract(r'(Touchscreen)')
    train['pantalla'] = train['pantalla'].replace(r'(Touchscreen)','', regex = True)
    train['tactil'] = train['tactil'].replace('Touchscreen', 1)
    train['tactil'] = train['tactil'].replace(np.nan, 0)
    train['pantalla'] = train['pantalla'].replace(r' ','', regex = True)
    train['pantalla'] = train['pantalla'].replace(r'^\s*$', np.nan, regex = True)
    train = train.drop("ScreenResolution",axis = 1)
    

extract_screen_features (df_test)
extract_screen_features (df_train)

C:\Users\pobla\AppData\Local\Temp\ipykernel_17864\1730820539.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['tactil'] = train['tactil'].replace('Touchscreen', 1)
C:\Users\pobla\AppData\Local\Temp\ipykernel_17864\1730820539.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['tactil'] = train['tactil'].replace('Touchscreen', 1)


In [2710]:
def process_ram_column(dataframe):
    dataframe['Ram'] = dataframe['Ram'].str.replace('GB', '').astype(float)
    
process_ram_column (df_train)
process_ram_column (df_test)

In [2711]:
def extract_from_storage(train):
    
    train['almacenamiento1'] = train['Memory']
    train['almacenamiento1'] = train['almacenamiento1'].str.replace('1.0TB','1TB', regex = True)
    train['almacenamiento1'] = train['almacenamiento1'].str.replace('1TB','1000GB')
    train['almacenamiento1'] = train['almacenamiento1'].str.replace('2TB','2000GB')
    train['almacenamiento1'] = train['almacenamiento1'].str.replace('GB','')
    train['almacenamiento2'] = train['almacenamiento1'].str.replace(r' ','')
    almacenamiento1 = []
    almacenamiento2 = []
    for i in train['almacenamiento2']:
        if len(re.findall(r'\+', i)) == 1: 
            # Double drive
            one = re.findall(r'(\w+)', i)
            almacenamiento1.append(one[0])
            almacenamiento2.append(one[1])
        else: 
            # Single drive
            one = re.findall(r'(\w+)', i)
            almacenamiento1.append(one[0])
            almacenamiento2.append('NaN')


    #extracting size and type of primary Memory
    storage1size = []
    storage1type = []
    for i in almacenamiento1:
        storage1type.append(re.findall(r'(\D\w+)', i)[0])
        storage1size.append(re.findall(r'(\d+)', i)[0])


    #extracting size and type of secondary Memory
    storage2size = []
    storage2type = []
    for i in almacenamiento2:
        if i != 'NaN':
            storage2type.append(re.findall(r'(\D\w+)',i)[0])
            storage2size.append(re.findall(r'(\d+)',i)[0])
        else:
            storage2type.append('NaN')
            storage2size.append(0)
    train['primarystorage_size'] = storage1size
    train['primarystorage_type'] = storage1type
    train['secondarystorage_size'] = storage2size
    train['secondarystorage_type'] = storage2type
    
    train["primarystorage_size"] = train["primarystorage_size"].astype(float)
    train["secondarystorage_size"] = train["secondarystorage_size"].astype(float)
    train = train.drop(['almacenamiento1','almacenamiento2','Memory'], axis = 1)
    train = train.replace({'NaN' : np.nan})
    


extract_from_storage (df_train)
extract_from_storage (df_test)

In [2712]:
def clean_weight(train):
    train['Peso'] = train['Weight'].str.replace('kg','')
    train['Peso'] = train['Peso'].astype(float)
    
    
clean_weight (df_train)
clean_weight (df_test)

In [2713]:
def manufacturer(train):
    train['cpu_manftr'] = train['Cpu'].str.extract(r'^(\w+)') 
    train['gpu_manftr'] = train['Gpu'].str.extract(r'^(\w+)') 
    

manufacturer (df_test)
manufacturer (df_train)

In [2714]:
df_test.columns

Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'Vel', 'resolucion', 'pantalla', 'tactil', 'almacenamiento1',
       'almacenamiento2', 'primarystorage_size', 'primarystorage_type',
       'secondarystorage_size', 'secondarystorage_type', 'Peso', 'cpu_manftr',
       'gpu_manftr'],
      dtype='object')

In [2715]:
# Lista original de características
features = ['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
            'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
            'Vel', 'resolucion', 'pantalla', 'tactil', 'almacenamiento1',
            'almacenamiento2', 'primarystorage_size', 'primarystorage_type',
            'secondarystorage_size', 'secondarystorage_type', 'Peso', 'cpu_manftr',
            'gpu_manftr']

# Características a eliminar
features_to_remove = ['Cpu', 'Ram', 'Gpu', 'resolucion', 'primarystorage_type']

# Eliminar las características de la lista
for feature in features_to_remove:
    if feature in features:
        features.remove(feature)

print("Lista de características después de eliminar elementos:", features)


Lista de características después de eliminar elementos: ['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Memory', 'OpSys', 'Weight', 'Vel', 'pantalla', 'tactil', 'almacenamiento1', 'almacenamiento2', 'primarystorage_size', 'secondarystorage_size', 'secondarystorage_type', 'Peso', 'cpu_manftr', 'gpu_manftr']


In [2764]:
df_train.head ()

,id,laptop_ID,Company,Product,TypeName,Inches,Cpu,Ram,Gpu,OpSys,Price_euros,Vel,resolucion,pantalla,tactil,primarystorage_size,primarystorage_type,secondarystorage_size,secondarystorage_type,Peso,cpu_manftr,gpu_manftr
0,268,0.872991,-0.591108,-0.420313,0.332819,0.417157,0.781400,-0.038512,-1.115147,0.189706,749.00,0.803330,-1.011988,0.623931,-0.415014,1.590654,-1.170610,-0.412813,0.416841,0.501808,0.214176,-1.829683
1,347,0.522034,-1.073731,1.432225,0.332819,0.417157,1.714008,-0.855315,-0.495120,0.189706,449.00,-2.315602,-1.011988,0.623931,-0.415014,1.590654,-1.170610,-0.412813,0.416841,-0.049145,0.214176,-0.252489
2,353,0.779402,2.304632,0.179699,0.332819,-1.212927,-0.399903,-0.038512,-0.326021,2.050052,1460.00,0.023597,0.012734,0.623931,-0.415014,-0.514318,0.723971,-0.412813,0.416841,-1.273484,0.214176,-0.252489
3,578,0.262066,-0.591108,-1.470334,-1.290340,0.417157,1.030096,1.595095,1.252231,0.189706,2868.99,0.998264,3.428474,-1.634104,-0.415014,-0.514318,0.723971,2.187800,-2.378444,3.654482,0.214176,1.324705
4,42,1.494314,-0.591108,-0.142807,1.144398,-1.779913,-0.275556,-0.038512,-0.326021,2.050052,1713.37,0.218530,0.012734,0.623931,2.409558,-0.514318,0.723971,-0.412813,0.416841,-1.181659,0.214176,-0.252489


In [2717]:
df_train = df_train.drop(columns=['almacenamiento1', 'almacenamiento2', 'ScreenResolution', 'Memory', 'Weight' ])
df_test = df_test.drop(columns=['almacenamiento1', 'almacenamiento2', 'ScreenResolution', 'Memory', 'Weight' ])

In [2718]:
df_train.columns

Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches', 'Cpu',
       'Ram', 'Gpu', 'OpSys', 'Price_euros', 'Vel', 'resolucion', 'pantalla',
       'tactil', 'primarystorage_size', 'primarystorage_type',
       'secondarystorage_size', 'secondarystorage_type', 'Peso', 'cpu_manftr',
       'gpu_manftr'],
      dtype='object')

In [2719]:
df_test.columns 

Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches', 'Cpu',
       'Ram', 'Gpu', 'OpSys', 'Vel', 'resolucion', 'pantalla', 'tactil',
       'primarystorage_size', 'primarystorage_type', 'secondarystorage_size',
       'secondarystorage_type', 'Peso', 'cpu_manftr', 'gpu_manftr'],
      dtype='object')

In [2720]:
df_train.columns

Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches', 'Cpu',
       'Ram', 'Gpu', 'OpSys', 'Price_euros', 'Vel', 'resolucion', 'pantalla',
       'tactil', 'primarystorage_size', 'primarystorage_type',
       'secondarystorage_size', 'secondarystorage_type', 'Peso', 'cpu_manftr',
       'gpu_manftr'],
      dtype='object')

In [2721]:
df_train.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     912 non-null    int64  
 1   laptop_ID              912 non-null    int64  
 2   Company                912 non-null    object 
 3   Product                912 non-null    object 
 4   TypeName               912 non-null    object 
 5   Inches                 912 non-null    float64
 6   Cpu                    912 non-null    object 
 7   Ram                    912 non-null    float64
 8   Gpu                    912 non-null    object 
 9   OpSys                  912 non-null    object 
 10  Price_euros            912 non-null    float64
 11  Vel                    912 non-null    float64
 12  resolucion             912 non-null    object 
 13  pantalla               259 non-null    object 
 14  tactil                 912 non-null    float64
 15  primar

In [2722]:
df_train.columns

Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches', 'Cpu',
       'Ram', 'Gpu', 'OpSys', 'Price_euros', 'Vel', 'resolucion', 'pantalla',
       'tactil', 'primarystorage_size', 'primarystorage_type',
       'secondarystorage_size', 'secondarystorage_type', 'Peso', 'cpu_manftr',
       'gpu_manftr'],
      dtype='object')

In [2723]:
df_train ['Price_euros'].value_counts ()

Price_euros
1099.00    12
1499.00    11
1799.00     9
899.00      9
369.00      8
999.00      8
1349.00     7
699.00      7
1199.00     7
389.00      6
629.00      6
949.00      6
799.00      6
349.00      6
549.00      6
1399.00     5
1149.00     5
299.00      5
1299.00     5
2299.00     5
459.00      5
979.00      5
1899.00     5
2799.00     4
819.00      4
1599.00     4
1195.00     4
1649.00     4
959.00      4
1749.00     4
209.00      4
479.00      4
499.00      4
649.00      4
309.00      4
689.00      4
779.00      4
2499.00     4
805.99      3
749.00      3
638.00      3
1129.00     3
839.00      3
399.00      3
449.00      3
889.00      3
398.99      3
977.00      3
1159.00     3
439.00      3
469.00      3
619.00      3
639.00      3
559.00      3
1379.00     3
249.00      3
289.00      3
599.00      3
1119.00     3
1975.00     3
579.00      3
199.00      2
919.00      2
429.00      2
720.32      2
409.00      2
1590.00     2
564.00      2
1269.00     2
597.00      2
589.00  

In [2724]:
cat_col = ['Company', 'Product', 'TypeName', 'Cpu',
       'Gpu', 'OpSys', 'resolucion',
       'pantalla', 'tactil', 'primarystorage_type',
       'secondarystorage_type', 'cpu_manftr',
       'gpu_manftr']

In [2725]:
df_train.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     912 non-null    int64  
 1   laptop_ID              912 non-null    int64  
 2   Company                912 non-null    object 
 3   Product                912 non-null    object 
 4   TypeName               912 non-null    object 
 5   Inches                 912 non-null    float64
 6   Cpu                    912 non-null    object 
 7   Ram                    912 non-null    float64
 8   Gpu                    912 non-null    object 
 9   OpSys                  912 non-null    object 
 10  Price_euros            912 non-null    float64
 11  Vel                    912 non-null    float64
 12  resolucion             912 non-null    object 
 13  pantalla               259 non-null    object 
 14  tactil                 912 non-null    float64
 15  primar

In [2726]:
df_test.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     391 non-null    int64  
 1   laptop_ID              391 non-null    int64  
 2   Company                391 non-null    object 
 3   Product                391 non-null    object 
 4   TypeName               391 non-null    object 
 5   Inches                 391 non-null    float64
 6   Cpu                    391 non-null    object 
 7   Ram                    391 non-null    float64
 8   Gpu                    391 non-null    object 
 9   OpSys                  391 non-null    object 
 10  Vel                    391 non-null    float64
 11  resolucion             391 non-null    object 
 12  pantalla               106 non-null    object 
 13  tactil                 391 non-null    float64
 14  primarystorage_size    391 non-null    float64
 15  primar

In [2727]:
df_train = df_train.fillna ( 'NaN')
df_test = df_test.fillna ( 'NaN')

In [2728]:
df_train.head (5)

,id,laptop_ID,Company,Product,TypeName,Inches,Cpu,Ram,Gpu,OpSys,Price_euros,Vel,resolucion,pantalla,tactil,primarystorage_size,primarystorage_type,secondarystorage_size,secondarystorage_type,Peso,cpu_manftr,gpu_manftr
0,268,1002,Dell,Inspiron 5567,Notebook,15.6,Intel Core i7 7500U,8.0,AMD Radeon R7 M445,Windows 10,749.00,2.7,1366x768,NaN,0.0,1000.0,HDD,0.0,NaN,2.36,Intel,AMD
1,347,867,Asus,X541NA (N4200/4GB/1TB/W10),Notebook,15.6,Intel Pentium Quad Core N4200,4.0,Intel HD Graphics 505,Windows 10,449.00,1.1,1366x768,NaN,0.0,1000.0,HDD,0.0,NaN,2.00,Intel,Intel
2,353,966,Toshiba,Portege Z30-C-1CW,Notebook,13.3,Intel Core i5 6200U,8.0,Intel HD Graphics 520,Windows 7,1460.00,2.3,1920x1080,NaN,0.0,256.0,SSD,0.0,NaN,1.20,Intel,Intel
3,578,767,Dell,Alienware 17,Gaming,15.6,Intel Core i7 7700HQ,16.0,Nvidia GeForce GTX 1070,Windows 10,2868.99,2.8,3840x2160,IPSPanel,0.0,256.0,SSD,1000.0,HDD,4.42,Intel,Nvidia
4,42,1241,Dell,Latitude E7270,Ultrabook,12.5,Intel Core i5 6300U,8.0,Intel HD Graphics 520,Windows 7,1713.37,2.4,1920x1080,NaN,1.0,256.0,SSD,0.0,NaN,1.26,Intel,Intel


In [2729]:

df_train_cols = ['Cpu', 'Gpu', 'resolucion', 'primarystorage_type']

In [2730]:
en = LabelEncoder ()
for cols in cat_col:
    df_train [cols] = en.fit_transform (df_train [cols])

print (df_train.shape)

(912, 22)


In [2731]:
df_test.columns

#cat_cols = ['Cpu', 'Gpu', 'resolucion', 'primarystorage_type']


Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches', 'Cpu',
       'Ram', 'Gpu', 'OpSys', 'Vel', 'resolucion', 'pantalla', 'tactil',
       'primarystorage_size', 'primarystorage_type', 'secondarystorage_size',
       'secondarystorage_type', 'Peso', 'cpu_manftr', 'gpu_manftr'],
      dtype='object')

In [2732]:
en = LabelEncoder ()
for cols in cat_col:
    df_test [cols] = en.fit_transform (df_test [cols])

print (df_test.shape)

(391, 21)


In [2733]:
df_test.head (5)

,id,laptop_ID,Company,Product,TypeName,Inches,Cpu,Ram,Gpu,OpSys,Vel,resolucion,pantalla,tactil,primarystorage_size,primarystorage_type,secondarystorage_size,secondarystorage_type,Peso,cpu_manftr,gpu_manftr
0,181,1098,6,193,4,13.3,48,16.0,34,4,2.7,12,0,0,512.0,3,0.0,1,1.30,1,2
1,708,330,0,25,3,15.6,2,8.0,19,4,2.7,0,2,0,256.0,3,0.0,1,2.20,0,0
2,862,1260,0,34,3,15.6,25,4.0,29,1,2.0,0,2,0,500.0,1,0.0,1,2.40,1,2
3,1064,1137,6,46,3,14.0,30,8.0,29,6,2.3,3,2,0,256.0,3,0.0,1,1.43,1,2
4,702,1015,6,43,3,13.3,35,8.0,34,4,2.5,3,0,0,256.0,3,0.0,1,1.34,1,2


In [2734]:
df_test.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     391 non-null    int64  
 1   laptop_ID              391 non-null    int64  
 2   Company                391 non-null    int32  
 3   Product                391 non-null    int32  
 4   TypeName               391 non-null    int32  
 5   Inches                 391 non-null    float64
 6   Cpu                    391 non-null    int32  
 7   Ram                    391 non-null    float64
 8   Gpu                    391 non-null    int32  
 9   OpSys                  391 non-null    int32  
 10  Vel                    391 non-null    float64
 11  resolucion             391 non-null    int32  
 12  pantalla               391 non-null    int32  
 13  tactil                 391 non-null    int64  
 14  primarystorage_size    391 non-null    float64
 15  primar

In [2735]:
df_train.head (5)

,id,laptop_ID,Company,Product,TypeName,Inches,Cpu,Ram,Gpu,OpSys,Price_euros,Vel,resolucion,pantalla,tactil,primarystorage_size,primarystorage_type,secondarystorage_size,secondarystorage_type,Peso,cpu_manftr,gpu_manftr
0,268,1002,4,178,3,15.6,64,8.0,24,5,749.00,2.7,0,2,0,1000.0,1,0.0,2,2.36,1,0
1,347,867,2,425,3,15.6,79,4.0,35,5,449.00,1.1,0,2,0,1000.0,1,0.0,2,2.00,1,1
2,353,966,16,258,3,13.3,45,8.0,38,7,1460.00,2.3,3,2,0,256.0,3,0.0,2,1.20,1,1
3,578,767,4,38,1,15.6,68,16.0,66,5,2868.99,2.8,13,0,0,256.0,3,1000.0,0,4.42,1,2
4,42,1241,4,215,4,12.5,47,8.0,38,7,1713.37,2.4,3,2,1,256.0,3,0.0,2,1.26,1,1


In [2736]:
df_test.columns

Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches', 'Cpu',
       'Ram', 'Gpu', 'OpSys', 'Vel', 'resolucion', 'pantalla', 'tactil',
       'primarystorage_size', 'primarystorage_type', 'secondarystorage_size',
       'secondarystorage_type', 'Peso', 'cpu_manftr', 'gpu_manftr'],
      dtype='object')

In [2737]:

# Supongamos que 'df_test' es tu conjunto de datos y 'y' es tu variable objetivo

# Lista de características numéricas que deseas estandarizar
numeric_features = ['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches', 'Cpu',
       'Ram', 'Gpu', 'OpSys', 'Vel', 'resolucion', 'pantalla', 'tactil',
       'primarystorage_size', 'primarystorage_type', 'secondarystorage_size',
       'secondarystorage_type', 'Peso', 'cpu_manftr', 'gpu_manftr']

#numeric_features = ['Ram']

# Excluir la columna 'Id' u otras columnas que no deseas escalar
if 'id' in numeric_features:
    numeric_features.remove('id')

# Seleccionar solo las columnas que deseas estandarizar
X_to_scale = df_test[numeric_features]

# Estandarizar solo las características seleccionadas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_to_scale)

# Crear un nuevo DataFrame con las características estandarizadas
X_scaled_df = pd.DataFrame(X_scaled, columns=numeric_features)

# Reemplazar las características estandarizadas en el DataFrame original
df_test[numeric_features] = X_scaled_df

In [2738]:
df_train.columns

Index(['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches', 'Cpu',
       'Ram', 'Gpu', 'OpSys', 'Price_euros', 'Vel', 'resolucion', 'pantalla',
       'tactil', 'primarystorage_size', 'primarystorage_type',
       'secondarystorage_size', 'secondarystorage_type', 'Peso', 'cpu_manftr',
       'gpu_manftr'],
      dtype='object')

In [2739]:

# Supongamos que 'df_train' es tu conjunto de datos y 'y' es tu variable objetivo

# Lista de características numéricas que deseas estandarizar
numeric_features = ['id', 'laptop_ID', 'Company', 'Product', 'TypeName', 'Inches', 'Cpu',
       'Ram', 'Gpu', 'OpSys', 'Price_euros', 'Vel', 'resolucion', 'pantalla',
       'tactil', 'primarystorage_size', 'primarystorage_type',
       'secondarystorage_size', 'secondarystorage_type', 'Peso', 'cpu_manftr',
       'gpu_manftr']

#numeric_features = ['Ram', 'Price_euros']

# Excluir la columna 'Id' u otras columnas que no deseas escalar
if 'id' in numeric_features:
    numeric_features.remove('id')
if 'Price_euros' in numeric_features:
    numeric_features.remove('Price_euros')

# Seleccionar solo las columnas que deseas estandarizar
X_to_scale = df_train[numeric_features]

# Estandarizar solo las características seleccionadas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_to_scale)

# Crear un nuevo DataFrame con las características estandarizadas
X_scaled_df = pd.DataFrame(X_scaled, columns=numeric_features)

# Reemplazar las características estandarizadas en el DataFrame original
df_train[numeric_features] = X_scaled_df

In [2740]:
df_train.head ()

,id,laptop_ID,Company,Product,TypeName,Inches,Cpu,Ram,Gpu,OpSys,Price_euros,Vel,resolucion,pantalla,tactil,primarystorage_size,primarystorage_type,secondarystorage_size,secondarystorage_type,Peso,cpu_manftr,gpu_manftr
0,268,0.872991,-0.591108,-0.420313,0.332819,0.417157,0.781400,-0.038512,-1.115147,0.189706,749.00,0.803330,-1.011988,0.623931,-0.415014,1.590654,-1.170610,-0.412813,0.416841,0.501808,0.214176,-1.829683
1,347,0.522034,-1.073731,1.432225,0.332819,0.417157,1.714008,-0.855315,-0.495120,0.189706,449.00,-2.315602,-1.011988,0.623931,-0.415014,1.590654,-1.170610,-0.412813,0.416841,-0.049145,0.214176,-0.252489
2,353,0.779402,2.304632,0.179699,0.332819,-1.212927,-0.399903,-0.038512,-0.326021,2.050052,1460.00,0.023597,0.012734,0.623931,-0.415014,-0.514318,0.723971,-0.412813,0.416841,-1.273484,0.214176,-0.252489
3,578,0.262066,-0.591108,-1.470334,-1.290340,0.417157,1.030096,1.595095,1.252231,0.189706,2868.99,0.998264,3.428474,-1.634104,-0.415014,-0.514318,0.723971,2.187800,-2.378444,3.654482,0.214176,1.324705
4,42,1.494314,-0.591108,-0.142807,1.144398,-1.779913,-0.275556,-0.038512,-0.326021,2.050052,1713.37,0.218530,0.012734,0.623931,2.409558,-0.514318,0.723971,-0.412813,0.416841,-1.181659,0.214176,-0.252489


-----------------------------------------------------------------------------------------------------------------

## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

In [2741]:
X = df_train.drop ('Price_euros', axis=1)
y = df_train ['Price_euros']

print (X.shape)
print (y.shape)

(912, 21)
(912,)


### 2. Dividir X_train, X_test, y_train, y_test

In [2742]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size= 0.2, random_state=42)

print (X_train.shape)
print (y_train.shape)

print (X_test.shape)
print (y_test.shape)

(729, 21)
(729,)
(183, 21)
(183,)


In [2743]:
X_train.info ()

<class 'pandas.core.frame.DataFrame'>
Index: 729 entries, 25 to 102
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     729 non-null    int64  
 1   laptop_ID              729 non-null    float64
 2   Company                729 non-null    float64
 3   Product                729 non-null    float64
 4   TypeName               729 non-null    float64
 5   Inches                 729 non-null    float64
 6   Cpu                    729 non-null    float64
 7   Ram                    729 non-null    float64
 8   Gpu                    729 non-null    float64
 9   OpSys                  729 non-null    float64
 10  Vel                    729 non-null    float64
 11  resolucion             729 non-null    float64
 12  pantalla               729 non-null    float64
 13  tactil                 729 non-null    float64
 14  primarystorage_size    729 non-null    float64
 15  primarysto

### 3. Crear y entrenar el modelo


In [2744]:
lr = LinearRegression ()
lr.fit(X_train,y_train)

LinearRegression()

In [2745]:
dtree = DecisionTreeRegressor()
dtree.fit(X_train,y_train)

DecisionTreeRegressor()

In [2746]:
lgbmr = LGBMRegressor()
lgbmr.fit(X_train,y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000153 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1044
[LightGBM] [Info] Number of data points in the train set: 729, number of used features: 21
[LightGBM] [Info] Start training from score 1122.043223
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

LGBMRegressor()

In [2747]:
#d.options.display.max_rows = None
#pd.options.display.max_columns = None

In [2748]:
X_train.info ()

<class 'pandas.core.frame.DataFrame'>
Index: 729 entries, 25 to 102
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     729 non-null    int64  
 1   laptop_ID              729 non-null    float64
 2   Company                729 non-null    float64
 3   Product                729 non-null    float64
 4   TypeName               729 non-null    float64
 5   Inches                 729 non-null    float64
 6   Cpu                    729 non-null    float64
 7   Ram                    729 non-null    float64
 8   Gpu                    729 non-null    float64
 9   OpSys                  729 non-null    float64
 10  Vel                    729 non-null    float64
 11  resolucion             729 non-null    float64
 12  pantalla               729 non-null    float64
 13  tactil                 729 non-null    float64
 14  primarystorage_size    729 non-null    float64
 15  primarysto

### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de MAE.

In [2749]:
lr_pred = lr.predict(X_test)
print("r2_score:",r2_score(y_test,lr_pred))

mae = mean_absolute_error(y_test, lr_pred)
print("mean_absolute_error (MAE):", mae)


mse = mean_squared_error(y_test, lr_pred)
print("mean_squared_error (MSE):", mse)


r2_score: 0.7286176256430309
mean_absolute_error (MAE): 247.2231939296811
mean_squared_error (MSE): 117211.70356676752


In [2750]:
dtree_pred = dtree.predict(X_test)

print("r2_score:",r2_score(y_test,dtree_pred))

r2_score: 0.7318768240654014


In [2751]:
lgbmr_pred = lgbmr.predict(X_test)
print("r2_score",r2_score(y_test,lgbmr_pred))

mae = mean_absolute_error(y_test, lgbmr_pred)
print("Mean Absolute Error (MAE):", mae)

r2_score 0.8318229675457841
Mean Absolute Error (MAE): 177.95387044946904


-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

In [2752]:
predicko = lgbmr.predict (df_test)


In [2753]:

# Especificar num_leaves
lgbm_model = LGBMRegressor(num_leaves=31)

# O ajustar max_depth
lgbm_model = LGBMRegressor(max_depth=6)  # Por ejemplo, si quieres un árbol con una profundidad máxima de 6

# Luego, procede con la creación y ajuste del modelo
lgbm_model.fit(X_train, y_train)

y_pred_target = best_model.predict(df_test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000162 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1044
[LightGBM] [Info] Number of data points in the train set: 729, number of used features: 21
[LightGBM] [Info] Start training from score 1122.043223
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

In [2754]:
predicko

array([2271.90355019,  685.7108952 ,  379.99103023, 1367.4501095 ,
       1181.40197303, 1493.8308258 , 2268.05737343,  596.24392454,
       2526.76291577, 1930.81226468, 1498.5435694 ,  596.35259969,
        195.6832321 ,  588.00544039,  459.76127558, 1309.53925904,
        431.47614104, 1645.84428768, 1492.28700628, 1361.29294898,
       1698.00753742, 1246.37941972, 1818.17606736, 1360.26043295,
       1680.24946935,  372.2882478 ,  839.43232733,  385.12922383,
       1390.64009625,  536.51471133, 1183.70163553, 1165.76407501,
       1763.05421185,  455.10868323,  619.23967941, 1575.24587297,
        917.93250569,  697.14823831, 2933.24583969,  436.3808545 ,
        592.96376644,  918.94824806, 1065.68876963, 1628.12038593,
       1106.01254595, 1817.63034391,  488.55479622, 2513.68266477,
        274.28824256, 1373.77725105, 1438.66087249,  962.05930373,
       1012.86360591,  391.69279368, 1297.33610575,  685.23766509,
       1138.4482255 ,  220.5776154 , 1265.75532979,  895.43503

In [2755]:
model = df_test.index

In [2756]:
final = pd.DataFrame ()
final_2 = final.copy ()

final.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [2757]:
final ['Id'] = df_test ['id']

final_2 ['Id'] = df_test ['id']


In [2758]:
final.head ()

,Id
0,181
1,708
2,862
3,1064
4,702


In [2759]:
final ['Price_euros'] = predicko
final_2 ['Price_euros'] = y_pred_target


In [2760]:
final.to_csv ('Sample_submission.csv', index=False)
final_2.to_csv ('Sample_submissioon.csv', index=False)

In [2761]:
final.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id           391 non-null    int64  
 1   Price_euros  391 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 6.2 KB


### 2. Carga los datos de `test.csv` para predecir.

**¿Por qué puede dar error?** 

IMPORTANTE: APLICAR A ESTOS DATOS LO MISMO QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [2762]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [2763]:
predictions = model.predict(X_pred)
predictions

AttributeError: 'RangeIndex' object has no attribute 'predict'

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [ ]:
sample = pd.read_csv("sample_submission.csv") # Esta es mi ruta del archivo, la vuestra puede ser otra

In [ ]:
sample.head()

,Id,Price_euros
0,649,1563.982243
1,761,1241.640184
2,545,965.726008
3,367,1068.628745
4,361,1625.024059


In [ ]:
sample.shape

(46, 2)

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [ ]:
submission = final

In [ ]:
import urllib.request

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [ ]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("")


In [ ]:
chequeator(submission) # submission es el nombre que le habríamos puesto a nuestro .csv con los valores que me salieron en la predicción

Check the number of rows and/or columns and try again



In [ ]:
submission.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id           391 non-null    int64  
 1   Price_euros  391 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 6.2 KB
